In [9]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from cvxpy import *

In [10]:
#Import the price matrix
Price = np.mat([range(1000),np.random.uniform(10,20,1000),np.arange(1.0,1.999,0.001)])
Price = Price.T
Price

matrix([[   0.        ,   13.86605221,    1.        ],
        [   1.        ,   14.54093031,    1.001     ],
        [   2.        ,   16.11664458,    1.002     ],
        ..., 
        [ 997.        ,   10.31566116,    1.997     ],
        [ 998.        ,   15.17463768,    1.998     ],
        [ 999.        ,   14.32408048,    1.999     ]])

In [11]:
#Calculate mu and V
def adapted_stats(Price,trade_date,horizon,sample_frequency,number_of_samples,rate_of_decay):
    h = horizon #num
    P = Price #mat
    t_d = trade_date #num
    s_f = sample_frequency #num
    n_s = number_of_samples #num
    r_d = rate_of_decay #num

    s_d = t_d-1-np.arange(n_s+1)*s_f #reverse chronological
    s_d = s_d[::-1] #chronological    
    #sample dates, a row vector

    S_P = P[s_d,:]
    #Sample Prices, a matrix

    S_C_R = np.log(S_P[1:,:]/S_P[:-1,:])
    #Sample Compound Returns, a matrix

    #now it ’s time to construct the weight
    w = (1-r_d)**np.arange(1,n_s+1)
    w = w[::-1]
    w = w/sum(w)
    wt = np.transpose([w])
    #weights, a non-negative vector that sums to 1

    mean_c_r = S_C_R.T*wt
    #mean vector of compound returns

    Cov_C_R = (S_C_R.T*np.diag(w)*S_C_R)-mean_c_r*np.transpose([mean_c_r])
    #covariance matrix of compound returns

    adapted_mean_c_r = mean_c_r*(h/s_f)
    #adapting mean vector to reflect length of holding period

    Adapted_Cov_C_R = Cov_C_R*(h/s_f)
    #adapting covariance matrix to length of holding period

    muu = np.exp(adapted_mean_c_r + 0.5*np.mat(np.diag(Adapted_Cov_C_R)).T)
    mu = muu - 1
    #resulting mean vector of ARITHMETIC returns

    V = np.multiply((muu*np.transpose([muu.T])),(np.exp(Adapted_Cov_C_R)-1))
    #resulting covariance matrix of ARITHMETIC returns

    return mu,V

In [12]:
mu,V = adapted_stats(Price,500,5,5,10,0.004)
mu,V

(matrix([[ 0.01061058],
         [ 0.01912234],
         [ 0.00339784]]),
 matrix([[  1.04681374e-07,  -3.23144383e-06,   1.07286128e-08],
         [ -3.23144383e-06,   8.00096506e-02,  -3.80096347e-07],
         [  1.07286128e-08,  -3.80096347e-07,   1.09991842e-09]]))

In [16]:
from cvxpy import *
import numpy as np
def markowitz(mu,V,sigma,xx,trans_cost,total_trans_cost):
    # Problem Data
    n = len(mu)
    U = np.linalg.cholesky(V)
    e = np.ones([n,1])
    # Construct the problem.
    x = Variable(n)
    y = Variable(n)
    objective = Maximize(mu.T*x)
    constraints = [quad_form(x, V)<=sigma**2,
                   sum_entries(x)==1,
                   x-y==xx,
                   trans_cost*sum_entries(abs(y))<=total_trans_cost,
                   max_entries(abs(x))<=0.5,]
    prob = Problem(objective, constraints)
    result = prob.solve(solver=ECOS)
    prob.solve()  # Returns the optimal value.
    print "status:", prob.status
    print "optimal value", prob.value
    print "optimal var", x.value, y.value
    return x.value,y.value

In [17]:
xx = np.zeros(3)
markowitz(mu,V,0.1,xx,0,50)

status: optimal
optimal value 0.0125636481668
optimal var [[ 0.5       ]
 [ 0.35355247]
 [ 0.14644753]] [[ 0.5       ]
 [ 0.35355247]
 [ 0.14644753]]


(matrix([[ 0.5       ],
         [ 0.35355247],
         [ 0.14644753]]), matrix([[ 0.5       ],
         [ 0.35355247],
         [ 0.14644753]]))

In [ ]:
def rebalance_benchmark(benchmark_x0,benchmark_x,trans_cost):
    n = length(benchmark_x);
    z = Variable(1)
    objective = Maximize(z)
    constraints = [(n+1)*z<=benchmark_x0+sum_entries(benchmark_x)-trans_cost*sum_entries(abs(z-benchmark_x))]
    prob = Problem(objective, constraints)
    result = prob.solve(solver=ECOS)
    prob.solve()
    return z.value

In [2]:
import numpy as np
sum(np.arange(4))



6

In [ ]:
#function  multi_period_hw2_complete

#load hw2.mat; 

n = np.size(Price)/len(Price) # n=number of risky assets
e = np.ones([n,1])

###############
#### PARAMETERS
#### (the following choices of parameters can easily be changed)

horizon = 4 # rebalance monthly (every 4 weeks)
start = 400 # the week in which you are first given a portfolio to rebalance
number_rebalances = 100 # the number of times the portfolio will be rebalanced 
number_of_samples = 100 # how many samples are to be used 
                        # in computing return avereages and covariances
sample_frequency = 2 # 1 = weekly, 2 = biweekly, etc.
r_w_f_o_y_e = 0.4 # "relative weight for one year earlier" 
                 # -- a value .4 means that for the (exponential) weights 
                 # used in computing return averages and covariances, 
                 # the weight assigned to the time period one year ago
                 # should be .4 times the weight assigned 
                 # to the most recent period.    	 
allowable_risk = 1
    # This is the level of risk relative to the benchmark portfolio,
    #   where risk is measured as standard deviation of portfolio returns.
    # Choosing this value to equal 1 means exactly the same amount of risk is allowed,
    # whereas choosing 2 means twice as much risk is allowed as the benchmark, and so on.
trans_cost = 0.005  # transaction cost
wealth = 10000 # initial wealth measured in dollars, including money invested in assets
               # (one dollar invested in an asset is considered as one dollar of wealth,
               #  even though in liquidating the asset, transaction costs would be paid)   
x0 = 0.3 # proportion of wealth in bank initially
x = (0.7/n)*e # proportions in risky assets initially

# Assume the benchmark portfolio is initally equal-weighted, with 1/(n+1) being the 
# proportion of wealth invested in each asset and in the bank.
#### END OF PARAMETERS
######################

rate_of_decay = 1 - r_w_f_o_y_e**(sample_frequency/52)
initial_wealth = wealth
benchmark_wealth = wealth
rebalance_dates = start + horizon*np.arange(number_rebalances)

for i = range(len(rebalance_dates)):
    trade_date = rebalance_dates[i]

    ###### REBALANCE YOUR PORTFOLIO AND PAY TRANSACTION COSTS ######
    # It is more natural to rebalance the benchmark portfolio later #

    mu,V = adapted_stats(Price,trade_date,horizon,sample_frequency,number_of_samples,rate_of_decay)
    #[mu,V] = stats(Price,trade_date,sample_frequency,number_of_samples,rate_of_decay)
    #mu0 = (1+.01*risk_free_rate(trade_date-1))^(horizon/52) - 1;
    
    benchmark_risk = sqrt(np.transpose([e])*V*e)/(n+1)  # there are n+1 financial instruments
                                                        # including the bank
    sigma = allowable_risk*benchmark_risk
    
    #xx0 = x0
    xx = x
    x =  markowitz(mu,V,sigma,xx,trans_cost,total_trans_cost)
    
    wealth = wealth*(x0 + sum(x))
        # This is the same thing as updating your wealth by subtracting
        # all transaction costs from the rebalancing.  Indeed, in rebalancing,
        # the proportion of your wealth going to trans costs is 1 - x0 - sum(x).  
        
    total = sum(x)
    #x0 = x0/total
    x = x/total
        # Rescaling x0 and x so that the sum is 1 (i.e., proportions of current wealth)
    
    ###### PROCEED TO END OF TIME PERIOD AND ACCOUNT FOR GAINS, LOSSES ######
    
    returns = (Price(trade_date+horizon-1,:)-Price(trade_date-1,:))./Price(trade_date-1,:)
        # vector of actual returns for risky assets (this is a row vector)
    
    multiplier = 1+returns*x
    wealth = multiplier*wealth
            # by leaving off the semicolon, you can watch how wealth changes as the program runs

    if(wealth<=0):
        break   # stops the program if bankruptcy occurs
                # Not needed for benchmark portfolio (because it is long only)
    
    #x0 = (1+mu0)*x0/multiplier;
    x = np.multiply(x,np.transpose(1+returns))/multiplier
    # these are the proportions of current wealth invested in assets
    
    # Now its time to rebalance the benchmark portfolio and pay transaction costs
    
    benchmark_x0 = (1+mu0)/(n+1);
    benchmark_x = (1+returns)/(n+1);
    # This gives how the equal-weighted portfolio has changed during the time period.
    # The initial unit of wealth has become  benchmark_x0 + sum(benchmark_x).
    % This new level of wealth needs to be distributed equally among the assets and bank.
    % The optimal amount z to put into each one is determined by the following function,
    % which finds the value z so as to minimize transaction costs
    
    z = rebalance_benchmark(benchmark_x0,benchmark_x,trans_cost);  	
	benchmark_wealth = benchmark_wealth*(n+1)*z;
		
	% Until the end of the next time period, 
	% think of the benchmark portfolio as having been rebalanced
	% with wealth divided equally, that is, the portion of wealth invested in
	% each asset and the bank is 1/(n+1).	
    
end

fprintf('your final bank account %f\n');
x0
fprintf('your final risky portfolio %f\n');
x

fprintf('your final wealth %f\n',wealth);
fprintf('benchmark final wealth %f\n',benchmark_wealth);

fprintf('your annualized rate of return %f\n', (wealth/initial_wealth)^(52/(horizon*number_rebalances))-1);
fprintf('benchmark annualized rate of return %f\n', (benchmark_wealth/initial_wealth)^(52/(horizon*number_rebalances))-1);